# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [76]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [5]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "Weather_Database/WeatherPy_Database.csv"

# Load the CSV file into a Pandas DataFrame

city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barrow,US,71.2906,-156.7887,28.42,69,40,19.57,scattered clouds
1,1,Qaanaaq,GL,77.4840,-69.3632,21.49,95,99,6.78,light snow
2,2,Goundi,TD,9.3627,17.3660,84.07,67,95,2.48,overcast clouds
3,3,Puerto Ayora,EC,-0.7393,-90.3518,73.36,90,94,7.00,overcast clouds
4,4,Nanning,CN,22.8167,108.3167,64.85,41,95,4.50,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [9]:
# Prompt the user to enter the minimum temperature criteria

min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [10]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
                                 
# Display sample data
preferred_cities_df.head(10) 

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Goundi,TD,9.3627,17.3660,84.07,67,95,2.48,overcast clouds
6,6,Awbari,LY,26.5921,12.7805,88.86,16,26,7.07,scattered clouds
9,9,Bathsheba,BB,13.2167,-59.5167,88.41,70,20,12.66,few clouds
12,12,Sibolga,ID,1.7427,98.7792,76.24,88,100,2.44,overcast clouds
16,16,Vaini,TO,-21.2000,-175.2000,75.36,94,20,13.80,few clouds
17,17,Arraial Do Cabo,BR,-22.9661,-42.0278,80.46,69,0,24.16,clear sky
18,18,Hambantota,LK,6.1241,81.1185,77.34,100,75,8.05,shower rain
19,19,Sikonge,TZ,-5.6333,32.7667,81.07,17,12,4.38,few clouds
22,22,Kapaa,US,22.0752,-159.3190,76.98,92,100,4.61,overcast clouds
23,23,Butaritari,KI,3.0707,172.7902,80.60,79,100,12.53,light rain


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [11]:
# determine if there are any empty rows
preferred_cities_df.count()

City_ID                219
City                   219
Country                219
Lat                    219
Lng                    219
Max Temp               219
Humidity               219
Cloudiness             219
Wind Speed             219
Current Description    219
dtype: int64

In [58]:
# # Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities_df = preferred_cities_df.dropna()

# # Display sample data
clean_travel_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Goundi,TD,9.3627,17.3660,84.07,67,95,2.48,overcast clouds
6,6,Awbari,LY,26.5921,12.7805,88.86,16,26,7.07,scattered clouds
9,9,Bathsheba,BB,13.2167,-59.5167,88.41,70,20,12.66,few clouds
12,12,Sibolga,ID,1.7427,98.7792,76.24,88,100,2.44,overcast clouds
16,16,Vaini,TO,-21.2000,-175.2000,75.36,94,20,13.80,few clouds
...,...,...,...,...,...,...,...,...,...,...
681,681,Impfondo,CG,1.6381,18.0667,79.95,72,68,3.06,broken clouds
682,682,Eirunepe,BR,-6.6603,-69.8736,81.19,86,92,2.08,overcast clouds
685,685,Bintulu,MY,3.1667,113.0333,77.41,100,20,0.00,few clouds
689,689,Omboue,GA,-1.5746,9.2618,77.47,89,100,9.22,overcast clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [60]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities_df[["City", "Country", "Max Temp","Current Description", "Lat", "Lng"]].copy()

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
2,Goundi,TD,84.07,overcast clouds,9.3627,17.3660
6,Awbari,LY,88.86,scattered clouds,26.5921,12.7805
9,Bathsheba,BB,88.41,few clouds,13.2167,-59.5167
12,Sibolga,ID,76.24,overcast clouds,1.7427,98.7792
16,Vaini,TO,75.36,few clouds,-21.2000,-175.2000
17,Arraial Do Cabo,BR,80.46,clear sky,-22.9661,-42.0278
18,Hambantota,LK,77.34,shower rain,6.1241,81.1185
19,Sikonge,TZ,81.07,few clouds,-5.6333,32.7667
22,Kapaa,US,76.98,overcast clouds,22.0752,-159.3190
23,Butaritari,KI,80.60,light rain,3.0707,172.7902


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [61]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Goundi,TD,84.07,overcast clouds,9.3627,17.3660,
6,Awbari,LY,88.86,scattered clouds,26.5921,12.7805,
9,Bathsheba,BB,88.41,few clouds,13.2167,-59.5167,
12,Sibolga,ID,76.24,overcast clouds,1.7427,98.7792,
16,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,
17,Arraial Do Cabo,BR,80.46,clear sky,-22.9661,-42.0278,
18,Hambantota,LK,77.34,shower rain,6.1241,81.1185,
19,Sikonge,TZ,81.07,few clouds,-5.6333,32.7667,
22,Kapaa,US,76.98,overcast clouds,22.0752,-159.3190,
23,Butaritari,KI,80.60,light rain,3.0707,172.7902,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [62]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [63]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
#     hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [64]:
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Goundi,TD,84.07,overcast clouds,9.3627,17.3660,
6,Awbari,LY,88.86,scattered clouds,26.5921,12.7805,استراحة الشارب العائلي
9,Bathsheba,BB,88.41,few clouds,13.2167,-59.5167,Atlantis Hotel
12,Sibolga,ID,76.24,overcast clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
16,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
17,Arraial Do Cabo,BR,80.46,clear sky,-22.9661,-42.0278,Pousada Porto Praia
18,Hambantota,LK,77.34,shower rain,6.1241,81.1185,Bungalow 63
19,Sikonge,TZ,81.07,few clouds,-5.6333,32.7667,Sikonge Ndawvukilo Lodge
22,Kapaa,US,76.98,overcast clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
23,Butaritari,KI,80.60,light rain,3.0707,172.7902,Isles Sunset Lodge


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [79]:
clean_hotel_df=hotel_df.dropna()
# Drop the rows where there is no Hotel Name.
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
6,Awbari,LY,88.86,scattered clouds,26.5921,12.7805,استراحة الشارب العائلي
9,Bathsheba,BB,88.41,few clouds,13.2167,-59.5167,Atlantis Hotel
12,Sibolga,ID,76.24,overcast clouds,1.7427,98.7792,Hotel Syariah CN Darussalam
16,Vaini,TO,75.36,few clouds,-21.2000,-175.2000,Keleti Beach Resort
17,Arraial Do Cabo,BR,80.46,clear sky,-22.9661,-42.0278,Pousada Porto Praia
...,...,...,...,...,...,...,...
681,Impfondo,CG,79.95,broken clouds,1.6381,18.0667,Auberge La Place de Jules
682,Eirunepe,BR,81.19,overcast clouds,-6.6603,-69.8736,Hotel novo Acre & Comercial Aguiar
685,Bintulu,MY,77.41,few clouds,3.1667,113.0333,900 Boutique INN
689,Omboue,GA,77.47,overcast clouds,-1.5746,9.2618,Hotel Olako


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [20]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [80]:
# Set the file name.
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")


### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [81]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [82]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]

['\n<dl>\n<dt>Hotel Name</dt><dd>استراحة الشارب العائلي</dd>\n<dt>City</dt><dd>Awbari</dd>\n<dt>Country</dt><dd>LY</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 88.86 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Atlantis Hotel</dd>\n<dt>City</dt><dd>Bathsheba</dd>\n<dt>Country</dt><dd>BB</dd>\n<dt>Current Weather</dt><dd>few clouds and 88.41 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Syariah CN Darussalam</dd>\n<dt>City</dt><dd>Sibolga</dd>\n<dt>Country</dt><dd>ID</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 76.24 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Keleti Beach Resort</dd>\n<dt>City</dt><dd>Vaini</dd>\n<dt>Country</dt><dd>TO</dd>\n<dt>Current Weather</dt><dd>few clouds and 75.36 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Pousada Porto Praia</dd>\n<dt>City</dt><dd>Arraial Do Cabo</dd>\n<dt>Country</dt><dd>BR</dd>\n<dt>Current Weather</dt><dd>clear sky and 80.46 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Bungalow 

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [83]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
6,26.5921,12.7805
9,13.2167,-59.5167
12,1.7427,98.7792
16,-21.2000,-175.2000
17,-22.9661,-42.0278
18,6.1241,81.1185
19,-5.6333,32.7667
22,22.0752,-159.3190
23,3.0707,172.7902
28,29.1614,16.1390


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [85]:
# Add a marker layer for each city to the map. 
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# YOUR CODE HERE
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

# Create a figure to add the Google map as a layer
# YOUR CODE HERE

# Display the figure containing the map
# YOUR CODE HERE

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# Set the file name.
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")